# Naive Bayes

In [5]:
import numpy as np
import pandas as pd

In [20]:
def fit(X_train,Y_train):
    result = {}
    result["total_data"] = len(Y_train)
    class_value = set(Y_train)
    for current_class in class_value:
        result[current_class] ={}
        current_class_rows = (Y_train == current_class)#this will give a true false(1 or 0) value np array as o/p
        X_train_current = X_train[current_class_rows]#this will get u all the rows of the current class
        num_features = X_train.shape[1]
        result[current_class]["total_count"] = current_class_rows.sum()
        for j in range(1,num_features+1):
            
            result[current_class][j-1]={}
            all_possible_values = set(X_train[:,j-1])
           # result[current_class][j]["distint_features"] = len(all_possible_values)
            for current_value in all_possible_values:
                result[current_class][j-1][current_value] = (X_train_current[:,j-1]==current_value).sum()
                
                
    return result
            

In [25]:
def probability(dictionary,x,current_class):
    output =np.log(dictionary[current_class]["total_count"])-np.log(dictionary["total_data"])
    
    num_features = len(dictionary[current_class].keys())-1
    for j in range(1,num_features+1):
        xj = x[j-1]
        count_current_class_with_value_xj = dictionary[current_class][j-1][xj]+1
        count_current_class = dictionary[current_class]["total_count"]+ len(dictionary[current_class][j-1].keys())
        output = output+(np.log(count_current_class_with_value_xj)-np.log(count_current_class))
    return output
    
    
    
    

In [31]:
def predictSinglePoint(dictionary,x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if current_class == "total_data":
            continue
        p_current_class = probability(dictionary,x,current_class)
        if (first_run == True or p_current_class>best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False    
    return best_class
            

In [9]:
def predict(dictionary,X_test):
    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary,x)
        y_pred.append(x_class)
    return y_pred
        
        
    

In [14]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5*second_limit
    for i in range(len(column)):
        if column[i]<first_limit:
            column[i]=0
        elif column[i]<second_limit:
            column[i]=1
        elif column[i]<third_limit:
            column[i]=2 
        else:
            column[i]=3
                
            
    return column
            
    

In [12]:
from sklearn import datasets
iris = datasets.load_iris()
X=iris.data
Y=iris.target


In [15]:
for i in range(X.shape[-1]):
    X[:,i]= makeLabelled(X[:,i])

In [16]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)

In [22]:
dictionary = fit(X_train,Y_train)

In [34]:
Y_pred = predict(dictionary,X_test)
#Y_pred

In [35]:
from sklearn.metrics import classification_report ,confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       0.94      1.00      0.97        16
          2       1.00      0.89      0.94         9

avg / total       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
